This code might save some resources in load_sqlite_db_to_dataframe function.

In [ ]:
# embeddings = list(map(float, embeddings_str.split(',')))

In [1]:
import logging
import ast
import sqlite3
import pandas as pd
import numpy as np

def load_sqlite_db_to_dataframe(db_path:str) -> pd.DataFrame:
    logging.basicConfig(filename='logs.log',
                        level=logging.INFO,
                        format='%(asctime)s:%(levelname)s:%(message)s')
    try:
        conn = sqlite3.connect(db_path)
        logging.info(f'Established SQLite connection with: {db_path}')
        query = "SELECT * FROM algorithms_table"
        df = pd.read_sql_query(query, conn)
        df['embeddings'] = df['embeddings'].apply(ast.literal_eval)
        df['embeddings'] = np.stack(df['embeddings'].to_numpy()).tolist()
        logging.info('Database loaded and embeddings converted')
    except Exception as e:
        logging.error(f"Error loading database: {e}")
        raise e
    finally:
        conn.close()
        logging.info('Database connection closed')
    return df

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarities(df_embeddings, single_embedding):
    # Ensure df_embeddings is a 2D numpy array
    embeddings_array = np.array(df_embeddings.tolist())

    single_embedding_reshaped = np.reshape(single_embedding, (1, -1))
    print(single_embedding_reshaped.shape)

    # Calculate cosine similarity and directly convert to pandas Series.
    similarities = cosine_similarity(embeddings_array, single_embedding_reshaped).flatten()
    return pd.Series(similarities)

In [3]:
from openai import OpenAI

client = OpenAI()

def embed_text(text):
    response = client.embeddings.create(
        input = text,
        model = "text-embedding-3-large"
    )
    print("Embedding created.")
    return response.data[0].embedding

In [4]:
import pandas as pd

def search_by_embeddings(dataframe, user_question, n=5):
    embeded_user_q = embed_text(user_question)
    print("embeded_user_q created.")
    
    similarity_df = calculate_cosine_similarities(dataframe["embeddings"], embeded_user_q)
    print("similarity_df created.")
    
    dataframe['similarities'] = similarity_df
    print("dataframe['similarities'] created.")
    print(dataframe.head(5))
    
    res = dataframe.sort_values('similarities', ascending=False).head(n)
    print(res.head(5))
    return res


In [5]:
import pandas as pd

df = load_sqlite_db_to_dataframe('/Users/jdo/EnBed/EnBed/db/EnBed.sqlite')  # Provide a clear path as an argument
search_query = """ 

Design an efficient algorithm to merge two priority queues. Analyze the problem, design an algorithm for merging the priority queues, and justify the algorithm’s design. The answer must use 'K' (defined below)!
##############NEW PAGE##############
**Grading Criteria**
- Q2 Analysis: Problem
  - 3 pts Full Marks: Problem is defined with useful information.
  - 1.5 pts Half Marks: Problem is only partially defined, or occluded by extraneous information.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Analysis: Assumptions
  - 3 pts Full Marks: Assumptions given are reasonable, and fill any ambiguity in the problem which is required for solution approach.
  - 1.5 pts Half Marks: Assumptions are not reasonable, or are incomplete.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Analysis: Metrics
  - 4 pts Full Marks: Metric(s) for evaluating the effectiveness of a solution are given, are reasonable, and are measurable.
  - 2 pts Half Marks: Metric(s) are given implicitly, or are not measurable.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Design: Alignment
  - 2 pts Full Marks: Algorithm addresses problem stated in Analysis.
  - 1 pts Half Marks: Algorithm, when using libraries defined in K, attempts to follow the definitions given in K but has syntax or logical problems.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Design: Clarity (Proper Pseudocode)
  - 3 pts Full Marks: Algorithm is well defined and contains all algorithmic thinking needed to implement it.
  - 1.5 pts Half Marks: Algorithm contains most of the algorithmic thinking needed to implement it, but relies on undefined external functions or is missing some core logic.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Design: Use of K
  - 2 pts Full Marks: Algorithm, when using libraries/functions) defined in K, follows the definitions given in K. (Slight variation ok, but must indicate use of K.)
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Justification: Alternatives
  - 4 pts Full Marks: Supports the selected design over alternatives. (Or shows that algorithm is at least as good as alternatives.)
  - 2 pts Half Marks: Supports the selected design over alternatives, but does not do so in a general or sound manner.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
- Q2 Justification: Alignment
  - 4 pts Full Marks: Justification is aligned with metrics from Analysis, does not omit any metrics, and evaluation of metrics is reasonable sound.
  - 2 pts Half Marks: Justification is aligned with metrics from Analysis, but either omits at least one metrics or does not soundly evaluate them.
  - 0 pts No Marks: Did not attempt, large logical jumps, or many writing errors that distract from readability, or contradicts K.
##############NEW PAGE##############
***K***
Let the background knowledge for solving this problem be:
**SER222: Priority Queues**
**Definition 1. Binary Tree:** A binary tree is a recursive data structure composed of elements called nodes that contain a value, and then references to at most two child nodes.
**Definition 2. Complete Binary Tree:** a complete binary tree is one where every level is full except the last, and the last level is filled from the left to the right.
**Definition 3. Maximum Heap:** “A binary tree is heap-ordered if each node is larger than or equal to the keys in that node’s two children (if any).” (Sedgewick). The term *key* refers to the label of a conceptual node in a heap, and the term *value* refers to some piece of data that is attached to that node. Any pair of keys may be compared to check their relative order.
**Definition 4. Heap-ordered Array:** We say that an array is heap-ordered if the root element of the heap (if it exists) is stored at index 1, and where the following formulas may be used to find a parent’s (call it *p*) left (call it *c_left*) and right children (call it *c_right*): *p = ⌊k/2⌋, c_left = 2k, c_right = 2k + 1*.
**Definition 5. Priority Queue:** a priority queue is an abstract data structure that supports adding (“insert”) and removing elements (“delMax”). Data is represented as a complete binary tree, and is stored as a heap-ordered array. Assume that keys are unique. Per the Sedgewick implementation, both operations take O(logn) time, and both result in a complete and heap-ordered tree. These times will be taken to be optimal. See Algorithm 1
"""

results = search_by_embeddings(df, search_query)
print(results)
print("Done")

Embedding created.
embeded_user_q created.
(1, 3072)
similarity_df created.
dataframe['similarities'] created.
   id                                  guid                            author  \
0   1  b4dd1587-d3ab-4556-991a-8c0e1111ca66  Robert Sedgewick and Kevin Wayne   
1   2  54b381fe-89b3-418b-be30-0ec9168cce74  Robert Sedgewick and Kevin Wayne   
2   3  da23c510-08d8-411f-b557-c761f74d44bc  Robert Sedgewick and Kevin Wayne   
3   4  24ff2b49-bc36-44b0-9c04-232fe1bd551a  Robert Sedgewick and Kevin Wayne   
4   5  0e1a0586-24f4-4ba4-b12b-2000bb3601ac  Robert Sedgewick and Kevin Wayne   

                        title  target_page page_range  \
0  Algorithms, Fourth Edition            1      0,1,2   
1  Algorithms, Fourth Edition            2      1,2,3   
2  Algorithms, Fourth Edition            3      2,3,4   
3  Algorithms, Fourth Edition            4      3,4,5   
4  Algorithms, Fourth Edition            5      4,5,6   

                                             summary  \
0  

In [6]:
# Assuming 'results' is the DataFrame with the relevant rows already selected
texts = results['text']
print(texts)

for i, text in enumerate(texts, 1):
    authority_info = f"Text {i}: {text}\n"
    print(authority_info)


331    <<<Page 319>>>\nProposition Q. In an N-key pri...
332    <<<Page 1>>>\n\nArray resizing. We can add a n...
329    is stronger than the new person, they exchange...
330    ```\nALGORITHM 2.6 Heap priority queue\npublic...
344    <<<Page 332>>>\n\nand M = 104. \n\n2.4.29 Min/...
Name: text, dtype: object
Text 1: <<<Page 319>>>
Proposition Q. In an N-key priority queue, the heap algorithms require no more than `1 + lg N` compares for insert and no more than `2 lg N` compares for remove the maximum. Proof: By Proposition P, both operations involve moving along a path between the root and the bottom of the heap whose number of links is no more than `lg N`. The remove the maximum operation requires two compares for each node on the path (except at the bottom): one to find the child with the larger key, the other to decide whether that child needs to be promoted. For typical applications that require a large number of intermixed insert and remove the maximum operations in a large prior

In [7]:
import os
import openai


client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
def question_master(question, augmented_knowledge):
    system_prompt_static = """
    You are a helpful assistant that answers questions in computer science and software engineering. 
    You always think step-by-step and think for a moment before answering.
    You have the following tasks:
    1. Process and understand the information contained in the 'augmented_knowledge' variable.
        The 'augmented_knowledge' variable is enclosed between the XML tags "<augmented_knowledge>" and "</augmented_knowledge>", 
        and contains the most relevant information from the course textbook.
    2. Consider information derived from the 'augmented_knowledge' variable as completely true and fully accurate.
    3. Read the user's question.
    4. Take the information derived from the 'augmented_knwoledge' and use it to answer the user's question.
    5. Answer the user's question to the best of your ability, unless you couldn't find the answer in 'augmented_knowledge' 
        in which case say "I couldn't find it, would you like me to look it up?".
    Deliverables: 
    1. The most important information that you found in the 'augmented_knowledge' variable.
    2. The answer to the user's question, which may include generating code or a detailed explanation. Many times, it will include answering a multiple choice question.
    """;
    system_prompt_dynamic = "Here is the most relevant information drawn from the course textbook: \n\n<augmented_knowledge>" + augmented_knowledge + "</augmented_knowledge>\n\n"
    system_prompt = system_prompt_static + system_prompt_dynamic
    response = client.chat.completions.create(
        # model="gpt-4",
        model="gpt-4-1106-preview",
        # model="gpt-3.5-turbo-0125",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    string_response = str(response.choices[0].message.content)
    return string_response


In [8]:
user_question = f'{search_query}'

In [9]:
answer = question_master(user_question, authority_info)
print(answer)

### Most Important Information from 'augmented_knowledge'

1. Min/max priority queue supports insert, delete max, delete min in logarithmic time, and find max, find min in constant time using two heaps.
2. Dynamic median-finding supports insert in logarithmic time, find median in constant time, and delete median in logarithmic time using a min-heap and a max-heap.
3. Fast insert in a MinPQ API with insert using ~log log N compares and delete the minimum using ~2 log N compares, suggesting the use of binary search on parent pointers in `swim()`.
4. It is impossible to develop a compare-based implementation of the MinPQ API with both insert and delete the minimum using ~N log log N compares.
5. Index priority-queue implementation modifies Algorithm 2.6 to maintain a binary heap using 1-based indexing, an inverse array, and an array of keys, with `qp[i]` = -1 if `i` is not on the queue.

### Answer to the User's Question

**Analysis: Problem**
The problem is to design an efficient algorit

In [10]:
with open('answer.txt', 'w') as f:
    f.write(answer)